<a href="https://colab.research.google.com/github/thomasp05/GIF-7005-Projet/blob/features%2Ftest-unet/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-configuration

## Mount Google Drive

In [ ]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Clone and pull github repository

In [1]:
# Make sure clone at root
%cd /content

!pip3 install pydicom
!git clone https://github.com/thomasp05/gif-705-projet

import os
os.chdir('gif-705-projet')

# Add the local_modules directory to the set of paths
# Python uses to look for imports.
import sys
sys.path.append('/content/gif-705-projet')

/content
fatal: destination path 'gif-705-projet' already exists and is not an empty directory.


Checkout branch

In [2]:
!git checkout "features/test-unet" #develop
!git pull

Already on 'features/test-unet'
Your branch is up to date with 'origin/features/test-unet'.
Already up to date.


# Imports and load

In [3]:
import time

import torch


from dataset import *

from models import *
import models_parts
from train_helper import *

from torch.optim import lr_scheduler


torch.manual_seed(111)

## HYPERPARAMETERS

In [4]:
N_EPOCH = 1
BATCH_SIZE = 4

In [10]:
google_drive_path = "/content/drive/My Drive/"

## Load dataset

In [11]:
dataset = dcm_dataset(google_drive_path + 'GIF-7005-Projet/gif-7005-projet/data', transforms=Downsample())
print("Found {} images".format(len(dataset.img_files)))

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, num_workers=2)

dataloaders = {
  'train': train_loader,
  'val': test_loader
}

Found 26684 images


# UNet

Instantiate Model

In [6]:
model = UNet(n_channels=1, n_classes=1).to("cuda:0")

## Simple Train

In [ ]:
# Test inference
optim = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

timer = time.time()
for epoch in range(N_EPOCH):
  for img, (target, bounding_box) in train_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    loss = criterion(out, bounding_box)
    loss.backward()

    optim.step()

  print("Epoch : {}".format(epoch+1))
  print("Time  : {:.2f}".format(time.time()-timer))

Epoch : 1
Time  : 4794.29


In [ ]:
checkpoint_path = google_drive_path + "unet_drive.pt"
torch.save(model.state_dict(), checkpoint_path)

In [ ]:
checkpoint_path = google_drive_path + "out_images_after_train.pt"
torch.save(out, checkpoint_path)

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()]) #


NameError: ignored

## Train

In [9]:
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.1)

checkpoint_path = google_drive_path + "unet_train_8_epochs.pt"

model = train_model(model, optimizer_ft, exp_lr_scheduler, dataloaders, 8, checkpoint_path)

Epoch 1/8
--------------------


KeyboardInterrupt: ignored

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
for img, (target, bounding_box) in test_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    break

plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()])


# ResNet UNet

Instantiate Model

In [ ]:
model = ResNetUNet(n_channels=1, n_classes=1).to("cuda:0")

Freeze backbone layers

In [ ]:
for l in model.base_layers:
  for param in l.parameters():
    param.requires_grad = False

## Simple Train

In [ ]:
# Test inference
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

timer = time.time()
for epoch in range(N_EPOCH):
  for img, (target, bounding_box) in train_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    loss = criterion(out, bounding_box)
    loss.backward()

    optim.step()

  print("Epoch : {}".format(epoch+1))
  print("Time  : {:.2f}".format(time.time()-timer))

Epoch : 1
Time  : 4794.29


In [ ]:
checkpoint_path = google_drive_path + "ResNetUNet_drive.pt"
torch.save(model.state_dict(), checkpoint_path)

In [ ]:
checkpoint_path = google_drive_path + "out_images_after_train_ResNetUNet.pt"
torch.save(out, checkpoint_path)

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()]) #


NameError: ignored

## Train

In [ ]:
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

checkpoint_path = google_drive_path + "intro_ml/unet_train_8_epochs.pt"

model = train_model(model, optimizer_ft, exp_lr_scheduler, dataloaders, 8, checkpoint_path)

Epoch 1/8
--------------------


KeyboardInterrupt: ignored

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
for img, (target, bounding_box) in test_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    break

plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()])
